# Part 1 - Predicting Review Scores on Pitchfork

For Part 1, we will be using data from [this paper](https://ojs.aaai.org/index.php/ICWSM/article/view/7355). The data is a collection of reviews from [Pitchfork](https://pitchfork.com/), a site that provides expert reviews of music album. The authors of this paper have also combined the data with a set of features from [Spotify’s API](https://developer.spotify.com/documentation/web-api/) that provide insight into the music itself, e.g. the "acousticness" of the song.  We will tackle a regression problem here, trying to predict the score of a review from several of the other columns in the dataset.

## Part 1.1 - Feature Engineering with Feature Subsets

In the first subsection of Part 1, we’re going to rely on our old friend linear regression. We’re going to look at how running linear regression with various subsets of our features impacts our ability to predict score.

In Part 1.1, your task is to write code below that trains a separate linear regression model for a number of different feature subsets.  Specifically:

- The list `feature_sets` below is a list of lists; each sublist is a different subset of features to build a model with. 
- All models should be trained on the dataset `part1_train.csv`. 
- For each of these trained models, you should evaluate the model’s predictions on the training dataset, as well as the provided test set, called `part1_test.csv`. The evaluation metric we will use is **root mean squared error**.  

Write out the result to a file called `part_1.1_results.csv` and submit this along with your assignment. The file should have the following columns:
- `feature_set` - a column describing the features of the model used. For feature sets with multiple features, combine them using an underscore (you can do this with the code `"_".join(feature_set)`)
- `training_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the training data
- `test_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the test data

In addition, please answer the following questions:
- **1.1.1** Which model had the best RMSE on the *training data*? 
- **1.1.2** Which model had the best RMSE on the *test data*? 
- **1.1.3** Which feature do you believe was the most important one? Why? *(Note: There is more than one perfectly acceptable way to answer this question)*
- **1.1.4** What can we say about the utility of the Spotify features based on these results?

In [ ]:
feature_sets = [['artist'],
 ['reviewauthor'],
 ['releaseyear'],
 ['recordlabel'],
 ['genre'],
 ['danceability'],
 ['energy'],
 ['key'],
 ['loudness'],
 ['speechiness'],
 ['acousticness'],
 ['instrumentalness'],
 ['liveness'],
 ['valence'],
 ['tempo'],
 ['danceability','energy','key','loudness','speechiness','acousticness',
  'instrumentalness','liveness','valence','tempo'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre', 'danceability', 
  'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
  'liveness', 'valence', 'tempo']]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Convenience things for you, note that releaseyear is continuous but is not a Spotify API variable
CONTINUOUS_FEATURES = ['releaseyear', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']
CATEGORICAL_FEATURES = ['artist', 'reviewauthor', 'recordlabel', 'genre']

#training Dataframe
training_data = pd.read_csv("part1_train.csv")
test_data = pd.read_csv("part1_test.csv")
for i in CATEGORICAL_FEATURES:
  cat_train=training_data[i]
  label=LabelEncoder()
  label.fit(cat_train)
  cat_train=pd.DataFrame(label.transform(cat_train))
  cat_train.columns=[i]
  cat_train.index=training_data.index
  training_data.drop(columns=i,inplace=True)
  training_data=training_data.join(cat_train,how='outer')

In [ ]:
#test dataframe
for i in CATEGORICAL_FEATURES:
  cat_test=test_data[i]
  label1=LabelEncoder()
  label1.fit(cat_test)
  cat_test=pd.DataFrame(label1.transform(cat_test))
  cat_test.columns=[i]
  cat_test.index=test_data.index
  test_data.drop(columns=i,inplace=True)
  test_data=test_data.join(cat_test,how='outer')
# print(test_data)

In [ ]:
from sklearn.linear_model import LinearRegression
result_data = dict()
f=[]
train=[]
test=[]

for feature in feature_sets:
    # Write your code for Part 1.1 here!
    x_train=training_data[feature]
    y_train=training_data['score']
    
    x_test= test_data[feature]
    y_test= test_data['score']

    #train RMSE
    LR=LinearRegression(fit_intercept=True).fit(x_train,y_train)
    y_pred_train=LR.predict(x_train)
    df_train=pd.DataFrame({'y_pred_train':y_pred_train,'y_train':y_train})
    train_RMSE=((df_train.y_pred_train - df_train.y_train)**2).mean() ** 0.5
    train.append(train_RMSE)

    #test RMSE
    LR1=LinearRegression(fit_intercept=True).fit(x_test,y_test)
    y_pred_test=LR1.predict(x_test)
    df_test=pd.DataFrame({'y_pred_test':y_pred_test,'y_test':y_test})
    test_RMSE=((df_test.y_pred_test - df_test.y_test)**2).mean() ** 0.5
    test.append(test_RMSE)

    if(len(feature)>1):
      feature="_".join(feature)
    else:
      feature=feature[0]

    f.append(feature)

result_data['feature_set']=f
result_data['training_rmse']=train
result_data['test_rmse']=test

final_file=pd.DataFrame(result_data)
final_file.to_csv('part_1.1_results.csv')

In [ ]:
#best RMSE on training data
f[train.index(min(train))]

'artist_reviewauthor_releaseyear_recordlabel_genre_danceability_energy_key_loudness_speechiness_acousticness_instrumentalness_liveness_valence_tempo'

In [ ]:
#best RMSE on test data
f[test.index(min(test))]

'artist_reviewauthor_releaseyear_recordlabel_genre_danceability_energy_key_loudness_speechiness_acousticness_instrumentalness_liveness_valence_tempo'

In [ ]:
#best feature
f[train.index(min(train[0:15]))]

'releaseyear'

## Part 1.2 - Feature Engineering with the LASSO

In Part 1.2, your task is to write code below that trains an L1-regularized linear regression model, with an expanded feature set.  Specifically:

1. Begin with the final feature set listed in `feature_sets` (i.e. your feature set, to begin this section, is `feature_sets[-1]`.
2. One-hot encode your categorical variables, setting `drop=if_binary` and `sparse=False` in the function arguments. 
3. Scale all of your continuous features using the `StandardScaler`.
4. Train an L1-regularized linear regression model using these features on the dataset `part1_train.csv`. You should use the [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) class in `sklearn`, it will do the cross-validation necessary to select the appropriate value for the regularizer for you!  Use 10-fold cross-validation to perform model selection (set the `LassoCV` parmaeter `cv` to 10), and set the `random_state` to 1. Do not change any of the other parameters to `LassoCV` (i.e. leave them at their defaults).
5. Identify the best `alpha` value (the regularizer term, according to `sklearn`. In class, we refer to this as $\lambda$!) in terms of average mean squared error according to the cross-validation.
6. Finally, train a [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) model on the entire training dataset (`part1_train.csv`). You will use this to report the root mean squared error on the test set (Question 1.2.4 below), and use it in Part 1.3 below as well.

**Hint: The proceedure outlined above is very similar to ones we have discussed in class and shown how to do using `Pipeline`s.** 


In [ ]:
# Write your code for Part 1.2 here
from sklearn.linear_model import LassoCV, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
CONTINUOUS_FEATURES = ['releaseyear', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']

# Do the CV to find alpha
training_data = pd.read_csv("part1_train.csv")
features = feature_sets[-1]
x_train_lassocv=training_data[features]
y_train_lassocv=training_data['score']
total_intro=0
for i in CATEGORICAL_FEATURES:
  datafra=x_train_lassocv[[i]]
  encoder_cv=OneHotEncoder(handle_unknown='ignore',drop='if_binary',sparse=False)
  datafra=pd.DataFrame(encoder_cv.fit_transform(datafra[[i]]))
  column = [ i+"_"+str(x) for x in encoder_cv.categories_[0]]
  total_intro+=len(column)
  datafra.columns=column
  datafra.index=x_train_lassocv.index
  x_train_lassocv.drop(columns=i,inplace=True)
  x_train_lassocv=x_train_lassocv.join(datafra,how='outer')    #training dataset

total_intro

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


680

In [ ]:
#rescaling continuous features
continuous_rescaled_x_lassocv=pd.DataFrame(StandardScaler().fit_transform(x_train_lassocv[CONTINUOUS_FEATURES].values))
continuous_rescaled_x_lassocv.columns=CONTINUOUS_FEATURES
continuous_rescaled_x_lassocv.index=x_train_lassocv.index
x_train_lassocv.drop(columns=CONTINUOUS_FEATURES,inplace=True)
x_train_lassocv=x_train_lassocv.join(continuous_rescaled_x_lassocv,how='outer')

In [ ]:
#training Lasso CV model
LCV=LassoCV(cv=10,random_state=1).fit(x_train_lassocv,y_train_lassocv)
#getting value of best alpha
best_alpha=LCV.alpha_
# len(LCV.alphas_)

In [ ]:
#lasso model on entire dataset
training_data = pd.read_csv("part1_train.csv")
test_data = pd.read_csv("part1_test.csv")

features_setting = ['releaseyear', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo','artist', 'reviewauthor', 'recordlabel', 'genre']
CONTINUOUS_FEATURES = ['releaseyear', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']
x_train_lasso=training_data[features_setting]
y_train_lasso=training_data['score']

x_test_lasso=test_data[features_setting]
y_test_lasso=test_data['score']

for i in CATEGORICAL_FEATURES:
  datafra=x_train_lasso[[i]]
  encoder2=OneHotEncoder(handle_unknown='ignore',drop='if_binary',sparse=False)
  encoder2.fit(datafra[[i]])
  datafra=pd.DataFrame(encoder2.transform(datafra[[i]]))
  column = [ i+"_"+str(x) for x in encoder2.categories_[0]]
  datafra.columns=column
  datafra.index=x_train_lasso.index
  x_train_lasso.drop(columns=i,inplace=True)
  x_train_lasso=x_train_lasso.join(datafra,how='outer')    #training dataset

  datafra_test=x_test_lasso[[i]]
  # encoder3=OneHotEncoder(handle_unknown='ignore',drop='if_binary',sparse=False)
  # encoder2.fit(datafra_test[[i]])
  datafra_test=pd.DataFrame(encoder2.transform(datafra_test[[i]]))
  column_test = [ i+"_"+str(x) for x in encoder2.categories_[0]]
  datafra_test.columns=column_test
  datafra_test.index=x_test_lasso.index
  x_test_lasso.drop(columns=i,inplace=True)
  x_test_lasso=x_test_lasso.join(datafra_test,how='outer')    #training dataset

#rescaling continuous features
continuous_rescaled_x_lasso=pd.DataFrame(StandardScaler().fit_transform(x_train_lasso[CONTINUOUS_FEATURES].values))
continuous_rescaled_x_lasso.columns=CONTINUOUS_FEATURES
continuous_rescaled_x_lasso.index=x_train_lasso.index
x_train_lasso.drop(columns=CONTINUOUS_FEATURES,inplace=True)
x_train_lasso=x_train_lasso.join(continuous_rescaled_x_lasso,how='outer')

continuous_rescaled_x_lasso_test=pd.DataFrame(StandardScaler().fit_transform(x_test_lasso[CONTINUOUS_FEATURES].values))
continuous_rescaled_x_lasso_test.columns=CONTINUOUS_FEATURES
continuous_rescaled_x_lasso_test.index=x_test_lasso.index
x_test_lasso.drop(columns=CONTINUOUS_FEATURES,inplace=True)
x_test_lasso=x_test_lasso.join(continuous_rescaled_x_lasso_test,how='outer')

lasso=Lasso(alpha=best_alpha)
lasso.fit(x_train_lasso,y_train_lasso)

y_pred_lasso_train=lasso.predict(x_train_lasso)
df_lasso_train=pd.DataFrame({'y_pred_lasso_train':y_pred_lasso_train,'y_train_lasso':y_train_lasso})
RMSE_train=((df_lasso_train.y_pred_lasso_train - df_lasso_train.y_train_lasso)**2).mean() ** 0.5

y_pred_lasso_test=lasso.predict(x_test_lasso)
df_lasso_test=pd.DataFrame({'y_pred_lasso_test':y_pred_lasso_test,'y_test_lasso':y_test_lasso})
RMSE_test=((df_lasso_test.y_pred_lasso_test - df_lasso_test.y_test_lasso)**2).mean() ** 0.5

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Now, answer the following questions:
- **1.2.1** - How many total features are introduced by Step 2 above? Provide both the number and an explanation of how you got to this number.
- **1.2.2** - What was the best `alpha` value according to your cross-validation results?
- **1.2.3** - What was the **average RMSE** of the model with this `alpha` value on the k-fold cross validation on the *training* data?
- **1.2.4** - What was the **RMSE** of the model with this `alpha` value on the k-fold cross validation on the *test* data?


In [ ]:
#1.2.1
total_intro

680

In [ ]:
#1.2.2
best_alpha

0.00016857012552111666

In [ ]:
#1.2.3
RMSE_train

1.1229212886742561

In [ ]:
#1.2.4
RMSE_test

1.1649956412612912

# Part 1.3 - Interpreting Model Coefficents

In this section we will interpret the coefficients from the final model you trained on all of the training data.

- **1.3.1** - How many non-zero coefficients are in this final model?
- **1.3.2** - What percentage of the coefficients are non-zero in this final model?
- **1.3.3** - Who were the three most critical review authors, as estimated by the model? How do you know?
- **1.3.4** - Who were the three artists that reviewers tended to like the most?  How do you know?
- **1.3.5** - What genre did Pitchfork reviewers tend to like the most? Which genre did they like the least?


In [ ]:
# Code for 1.3 here
#1.3.1
c=0
for i in lasso.coef_:
  if(i!=0):
    c+=1
c

484

In [ ]:
#1.3.2
percent=(c/len(lasso.coef_))*100
percent

70.04341534008684

In [ ]:
#1.3.3
d_author=dict()
for i in range(len(lasso.feature_names_in_)):
  if(lasso.feature_names_in_[i][0:12]=='reviewauthor'):
    d_author[lasso.feature_names_in_[i][13:]]=lasso.coef_[i]
d_author_new=sorted(d_author.items(), key=lambda x: x[1], reverse=False)
for i in range(3):
  print(d_author_new[i][0])

kris ex
Judy Berman
Jenn Pelly


In [ ]:
d_artist=dict()
for i in range(len(lasso.feature_names_in_)):
  if(lasso.feature_names_in_[i][0:6]=='artist'):
    d_artist[lasso.feature_names_in_[i][7:]]=lasso.coef_[i]
d_artist_new=sorted(d_artist.items(), key=lambda x: x[1], reverse=True)
for i in range(3):
  print(d_artist_new[i][0])

R.E.M.
Miles Davis
Kanye West


In [ ]:
d_genre=dict()
for i in range(len(lasso.feature_names_in_)):
  if(lasso.feature_names_in_[i][0:5]=='genre'):
    d_genre[lasso.feature_names_in_[i][6:]]=lasso.coef_[i]
d_genre_new=sorted(d_genre.items(), key=lambda x: x[1], reverse=True)
print("Most: ",d_genre_new[0][0])
print("Least: ",d_genre_new[-1][0])

Most:  Global
Least:  nan


# Part 1.4 - "Manual" Cross-Validation + Holdout for Model Selection and Evaluation

We will finally use cross validation for both algorithm and model selection, with a hold-out test set for a final evaluation. We will use **5-fold cross validation** to identify the best parameters and hyperparameters for a set of models. We will then take our final models and use a final hold-out test set (the same one as above) to estimate the generalization error of the models.

Specifically, your task is first to write code that trains and evaluates the following models, one for each of the specified hyper parameters sets:

- `Decision Tree regression` - All combinations of a `max_depth` of 5, 10, or 20, and a `criterion` of `"squared error"` or `"absolute error"`. Use the [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor).
- Ridge regression - Use the following choices of L2 penalty: $[10^{-5}, 10^{-4}, ..., 10^4, 10^5]$. In Python, you can create a list of these numbers using `np.logspace(-5, 5, 11)`. Use the [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) class from sklearn to train a Ridge Regression model. The parameters you need to pass when constructing the Ridge model are `alpha`, which lets you specify what you want the L2 penalty to be, and `random_state=0` to avoid randomness.
- kNN regression - Values of `n_neighbors` of 1, 5, 10, and 15. Use the [KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) class.

Additional notes:
1. All models should use the feature sets described in Part 1.3 (the same ...one-hot encoded... categorical variables, and the scaled continuous variables)
2. As opposed to using the `KFold` class from `sklearn` like we did in class, we have instead provided you with pre-existing data sets; you should therefore use the pre-split data in the provided CSV files ``1.2_fold0.csv ... 1.2_fold4.csv``

**What to submit**:

1. Write out the result to a file called `part_1.4_results.csv` and submit this along with your assignment. The file should have the following columns:
- `model_name` - The name of the model, one of `DTR` (Decision Tree Regression), `Ridge`, or `KNN`.
- `hyperparameter_setting` - a column describing the hyperparameters of the model. For models with multiple hyperparameters, combine them using an underscore (you can do this with the code `"_".join(hyperparameters)`).
- `mean_training_rmse` - a column that gives the mean RMSE on the k-fold training data. You should take the average of the model’s errors on the different folds, using root mean squared error again as your evaluation metric.
- `sd_training_rmse` - a column that gives the standard deviation RMSE on the k-fold training data.
- `test_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the test data


2. Answer the following questions:
- **1.4.1** Report, for each model, the hyper parameter setting that resulted in the best performance
- **1.4.2** Which model performed the best overall on the cross-validation? 
- **1.4.3** Which model performed the best overall on the final test set? 
- **1.4.4** With respect to your answer for 1.4.3, why do you think that might be? (*Note: there is more than one correct way to answer this question*)
- **1.4.5** Which model/hyperparameter setting had the highest standard deviation across the different folds of the cross validation?
- **1.4.6** With respect to your answer for 1.4.6, why do you think that might be? (*Note: there is more than one correct way to answer this question*)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
CONTINUOUS_FEATURES = ['releaseyear', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']
CATEGORICAL_FEATURES = ['artist', 'reviewauthor', 'recordlabel', 'genre']
training_data = pd.read_csv("part1_train.csv")
test_data = pd.read_csv("part1_test.csv")

In [ ]:
f1=pd.read_csv('1.2_fold0.csv')
f2=pd.read_csv('1.2_fold1.csv')
f3=pd.read_csv('1.2_fold2.csv')
f4=pd.read_csv('1.2_fold3.csv')
f5=pd.read_csv('1.2_fold4.csv')
l=['releaseyear', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo','artist', 'reviewauthor', 'recordlabel', 'genre']
list_df=[f2,f3,f4,f5]
# for i in list_df:
#   f1=f1.append(i)
f=pd.concat((f1,f2,f3,f4,f5),axis=0,ignore_index=True)
df=f

# y=f['score']
for i in CATEGORICAL_FEATURES:
    datafra=df[[i]]
    encod=OneHotEncoder(handle_unknown='ignore',drop='if_binary',sparse=False)
    encod.fit(datafra[[i]])
    datafra=pd.DataFrame(encod.transform(datafra[[i]]))
    column = [i+"_"+str(x) for x in encod.categories_[0]]
    datafra.columns=column
    datafra.index=df.index
    df.drop(columns=i,inplace=True)
    df=df.join(datafra,how='outer')    #training dataset

    datafra_test=test_data[[i]]
    encod_test=OneHotEncoder(handle_unknown='ignore',drop='if_binary',sparse=False)
    encod_test.fit(datafra_test[[i]])
    datafra_test=pd.DataFrame(encod_test.transform(datafra_test[[i]]))
    column = [i+"_"+str(x) for x in encod_test.categories_[0]]
    datafra_test.columns=column
    datafra_test.index=test_data.index
    test_data.drop(columns=i,inplace=True)
    test_data=test_data.join(datafra_test,how='outer')    #training dataset

#rescaling continuous features
continuous_rescaled_df=pd.DataFrame(StandardScaler().fit_transform(df[CONTINUOUS_FEATURES].values))
continuous_rescaled_df.columns=CONTINUOUS_FEATURES
continuous_rescaled_df.index=df.index
df.drop(columns=CONTINUOUS_FEATURES,inplace=True)
df=df.join(continuous_rescaled_df,how='outer')

continuous_rescaled_test_df=pd.DataFrame(StandardScaler().fit_transform(test_data[CONTINUOUS_FEATURES].values))
continuous_rescaled_test_df.columns=CONTINUOUS_FEATURES
continuous_rescaled_test_df.index=test_data.index
test_data.drop(columns=CONTINUOUS_FEATURES,inplace=True)
test_data=test_data.join(continuous_rescaled_test_df,how='outer')

# test_frame=pd.read_csv('part1_test.csv')

In [ ]:
f1=df.iloc[0:3254,:]
f2=df.iloc[3254:6508,:]
f2.reset_index(inplace=True)
f2.drop(columns='index',inplace=True)
f3=df.iloc[6508:9762,:]
f3.reset_index(inplace=True)
f3.drop(columns='index',inplace=True)
f4=df.iloc[9762:13016,:]
f4.reset_index(inplace=True)
f4.drop(columns='index',inplace=True)
f5=df.iloc[13016:16270,:]
f5.reset_index(inplace=True)
f5.drop(columns='index',inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
list_of_folds=[f1,f2,f3,f4,f5]
penalty=list(np.logspace(-5,5,11))
score_ridge=dict()
# score_kn2 = []
# test_rmse1=[]
for i in penalty:
  score_ridge[i]=list()
for i in range(len(list_of_folds)):
  test=list_of_folds[i]
  train=pd.DataFrame(data=None)
  for j in range(len(list_of_folds)):
    if(j!=i):
      train=pd.concat((train,list_of_folds[j]),axis=0,ignore_index=True)
  x_train=train.iloc[:,1:692]
  y_train=train['score']
  x_test=test.iloc[:,1:692]
  y_test=test['score']
  for k in penalty:
    list_penalty=list()
    rid=Ridge(alpha=k,random_state=0)
    rid.fit(x_train,y_train)
    y_pred=rid.predict(x_test)
    df_rid=pd.DataFrame({'y_pred':y_pred,'y_test':y_test})
    RMSE_ridge=((df_rid.y_pred - df_rid.y_test)**2).mean() ** 0.5
    score_ridge[k].append(RMSE_ridge)
avg_dic=dict()
for i in score_ridge:
  avg_dic[i]=[np.mean(score_ridge[i]),np.std(score_ridge[i])]
min_train_rmse=avg_dic[penalty[0]][0]
print(avg_dic)
h=[]
for i in avg_dic:
  h.append(avg_dic[i][0])
  if min_train_rmse > avg_dic[i][0]:
    min_train_rmse=avg_dic[i][0]
    std_train=avg_dic[i][1]
    hyp_ridge=i


rid1=Ridge(alpha=hyp_ridge,random_state=0)
rid1.fit(test_data.iloc[:,1:692],test_data['score'])
y_test_pred=rid1.predict(test_data.iloc[:,1:692])
df_rid=pd.DataFrame({'y_test_pred':y_test_pred,'y_test_data':test_data['score']})
test_RMSE_rid=((df_rid.y_test_pred - df_rid.y_test_data)**2).mean() ** 0.5

fin_ridge=pd.DataFrame({'model_name':'Ridge','hyperparameter_setting':hyp_ridge,'mean_training_rmse':min_train_rmse,'sd_training_rmse':std_train,'test_rmse':test_RMSE_rid},index=[0])
fin_ridge


# hyp_ridge=list(score_ridge.keys())[list(score_ridge.values()).index(min(score_ridge.values()))]

{1e-05: [1.1714285910726354, 0.023932410380583927], 0.0001: [1.171428112568338, 0.023932479762893505], 0.001: [1.1714233291446783, 0.023933173074720548], 0.01: [1.1713756550693613, 0.02394005651442725], 0.1: [1.1709135389972432, 0.02400501235344827], 1.0: [1.1673006710908447, 0.02451359925171606], 10.0: [1.160155236460384, 0.026582927824394545], 100.0: [1.1785917188719943, 0.027780491031218463], 1000.0: [1.2092672981319672, 0.027738128223743753], 10000.0: [1.2260474060740465, 0.028368540894899673], 100000.0: [1.2377235925796062, 0.028978982225486748]}


,model_name,hyperparameter_setting,mean_training_rmse,sd_training_rmse,test_rmse
0,Ridge,10.0,1.160155,0.026583,1.080606


In [ ]:
neigh=[1,5,10,15]
score_kn=dict()
for i in neigh:
  score_kn[i]=list()
for i in range(len(list_of_folds)):
  test=list_of_folds[i]
  train=pd.DataFrame(data=None)
  for j in range(len(list_of_folds)):
    if(j!=i):
      train=pd.concat((train,list_of_folds[j]),axis=0,ignore_index=True)
  x_train=train.iloc[:,1:692]
  y_train=train['score']
  x_test=test.iloc[:,1:692]
  y_test=test['score']
  for k in neigh:
    kn=KNeighborsRegressor(n_neighbors=k)
    kn.fit(x_train,y_train)
    y_pred=kn.predict(x_test)
    df_kn=pd.DataFrame({'y_pred':y_pred,'y_test':y_test})
    RMSE_kn=((df_kn.y_pred - df_kn.y_test)**2).mean() ** 0.5
    score_kn[k].append(RMSE_kn)

avg_dic=dict()
for i in score_kn:
  avg_dic[i]=[np.mean(score_kn[i]),np.std(score_kn[i])]
min_train_rmse=avg_dic[neigh[0]][0]
# print(avg_dic)
h=[]
for i in avg_dic:
  h.append(avg_dic[i][0])
  if min_train_rmse > avg_dic[i][0]:
    min_train_rmse=avg_dic[i][0]
    std_train=avg_dic[i][1]
    hyp_knn=i

knn1=KNeighborsRegressor(n_neighbors=hyp_knn)
knn1.fit(test_data.iloc[:,1:692],test_data['score'])
y_test_pred=knn1.predict(test_data.iloc[:,1:692])
df_knn=pd.DataFrame({'y_test_pred':y_test_pred,'y_test_data':test_data['score']})
test_RMSE_rid=((df_knn.y_test_pred - df_knn.y_test_data)**2).mean() ** 0.5

fin_knn=pd.DataFrame({'model_name':'KNN','hyperparameter_setting':hyp_knn,'mean_training_rmse':min_train_rmse,'sd_training_rmse':std_train,'test_rmse':test_RMSE_rid},index=[0])
fin_knn

# hyp_kn=list(score_kn.keys())[list(score_kn.values()).index(min(score_kn.values()))]

,model_name,hyperparameter_setting,mean_training_rmse,sd_training_rmse,test_rmse
0,KNN,15,1.206184,0.020578,1.155552


In [ ]:
list_of_folds=[f1,f2,f3,f4,f5]
score_tree_square=dict()
score_tree_abs=dict()
for i in range(5,20,5):
  score_tree_square[i]=list()
  score_tree_abs[i]=list()

for i in range(len(list_of_folds)):
  test=list_of_folds[i]
  train=pd.DataFrame(data=None)
  for j in range(len(list_of_folds)):
    if(j!=i):
      train=pd.concat((train,list_of_folds[j]),axis=0,ignore_index=True)
  x_train=train.iloc[:,1:692]
  y_train=train['score']
  x_test=test.iloc[:,1:692]
  y_test=test['score']
  for k in range(5,20,5):
    reg_square=DecisionTreeRegressor(max_depth=k,criterion="squared_error")
    reg_square.fit(x_train,y_train)
    y_pred=reg_square.predict(x_test)
    df_square=pd.DataFrame({'y_pred':y_pred,'y_test':y_test})
    RMSE_square=((df_square.y_pred - df_square.y_test)**2).mean() ** 0.5
    score_tree_square[k].append(RMSE_square)

    reg_abs=DecisionTreeRegressor(max_depth=k,criterion="absolute_error")
    reg_abs.fit(x_train,y_train)
    y_pred=reg_abs.predict(x_test)
    df_abs=pd.DataFrame({'y_pred':y_pred,'y_test':y_test})
    RMSE_abs=((df_abs.y_pred - df_abs.y_test)**2).mean() ** 0.5
    score_tree_abs[k].append(RMSE_abs)


In [ ]:
score_tree_abs

{5: [1.2089758551418306,
  1.2034628508748295,
  1.2595837279805926,
  1.2404397776758704,
  1.2174423172961657],
 10: [1.2205444667560574,
  1.243357272841843,
  1.2697405592843873,
  1.2729784098564516,
  1.2645456705550355],
 15: [1.2655298357652613,
  1.2669153577343615,
  1.3331245675018357,
  1.325919738537143,
  1.3030913318158002]}

In [ ]:
avg_dic_square=dict()
avg_dic_abs=dict()
for i in score_tree_square:
  avg_dic_square[i]=[np.mean(score_tree_square[i]),np.std(score_tree_square[i])]
  avg_dic_abs[i]=[np.mean(score_tree_abs[i]),np.std(score_tree_abs[i])]
# avg_dic_abs

In [ ]:
new=dict()
for i in avg_dic_square:
  if(avg_dic_square[i][0]<avg_dic_abs[i][0]):
    new[str(i)+"_squared_error"]=avg_dic_square[i]
  else:
    new[str(i)+"_absolute_error"]=avg_dic_abs[i]

new

{'10_squared_error': [1.2482628588566687, 0.02151649716051654],
 '15_absolute_error': [1.2989161662708804, 0.028479952418731858],
 '5_squared_error': [1.206996612035179, 0.023953895644957226]}

In [ ]:
min_train_rmse=new[list(new.keys())[0]][0]
# print(min_train_rmse)
std_train=new[list(new.keys())[0]][1]
hyp_dtr=list(new.keys())[0]
for i in new:
  # print(new[i][0])
  # h.append(avg_dic[i][0])
  if min_train_rmse > new[i][0]:
    min_train_rmse=new[i][0]
    std_train=new[i][1]
    hyp_dtr=i

In [ ]:
depth=int(hyp_dtr[0])
crit=hyp_dtr[2:]
dtr1=DecisionTreeRegressor(max_depth=depth,criterion=crit)
dtr1.fit(test_data.iloc[:,1:692],test_data['score'])
y_test_pred=dtr1.predict(test_data.iloc[:,1:692])
df_dtr=pd.DataFrame({'y_test_pred':y_test_pred,'y_test_data':test_data['score']})
test_RMSE_dtr=((df_dtr.y_test_pred - df_dtr.y_test_data)**2).mean() ** 0.5

fin_dtr=pd.DataFrame({'model_name':'DTR','hyperparameter_setting':hyp_dtr,'mean_training_rmse':min_train_rmse,'sd_training_rmse':std_train,'test_rmse':test_RMSE_dtr},index=[0])
fin_dtr

,model_name,hyperparameter_setting,mean_training_rmse,sd_training_rmse,test_rmse
0,DTR,5_squared_error,1.206997,0.023954,1.141591


In [ ]:
final_dataframe=pd.concat((fin_knn,fin_ridge,fin_dtr),axis=0,ignore_index=True)
final_dataframe.to_csv('part_1.4_results.csv')

# Part 2

In class, we have shifted from regression to classification. Here, we're going to get a little practice in optimizing one of the classification models we saw in class - logistic regression. As a reminder...

The loss function of logistic regression (also known as the logistic-loss or log-loss) is given by:
\begin{equation}
  J({\bf w}) = \frac{1}{n}\sum_{i=1}^n \log{(1 + \exp{(-y_i{\bf w}^\top{\bf x}_i}))}
  \label{eqn:logloss}
\end{equation}

The gradient for this loss function, as derived in class, is:
\begin{equation}
  \nabla J({\bf w}) = -\frac{1}{n}\sum_{i=1}^n \frac{y_i}{1 + \exp{(y_i{\bf w}^\top{\bf x}_i)}}{\bf x}_i
  \label{eqn:loglossgradient}
\end{equation}


The Hessian for the loss function is given by:
\begin{equation}
  {\bf H}({\bf w}) = \frac{1}{n} \sum_{i=1}^n \frac{\exp{(y_i{\bf w}^\top{\bf x}_i)}}{(1 + \exp{(y_i{\bf w}^\top{\bf x}_i)})^2}{\bf x}_i{\bf x}_i^\top
  \label{eqn:loglosshessian}
\end{equation}

## Part 2.1 - Logistic Regression with Gradient Descent

In Part 2.1 we will implement logistic regression with gradient descent. You need to finish implementing 3 functions:

1. `logistic_objective` - compute the logistic loss for the given data set (see equation above)
2. `logistic_gradient` - compute the gradient vector of logistic loss for the given data set (see equation above)
3. `run_gradient_descent` - run the gradient descent algorithm, given these two functions.

We have provided you with some simulation data to evaluate your method with. Part 2.1 will, however, largely be graded by evaluating your code on a slightly different dataset to ensure robustness. 

In addition, please submit answers to the following questions on your written report:

- **2.1.1** - How did you go about selecting a good step size, i.e. one that was not too big or too small? (*Note: There is more than one correct answer to this*)
- **2.1.2** - What is the condition under which we assume that the gradient descent algorithm has converged in the code here?
- **2.1.3** - What is a different convergence metric we could have used? (*Note: There is more than one correct answer to this*)

In [ ]:
import numpy as np
import math as math
import pandas as pd
import time


def logistic_objective(w, X, y):

    # compute log-loss error (scalar) with respect
    # to w (vector) for the given data X and y                               
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = scalar
    
    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    error = 0
    #extract the value of N
    n = len(X)
      
    for i in range(n):
        under_exp = -(y[i]*np.dot(X[i,:],w))
        temp1 = 1+ np.exp(under_exp)
        temp2 = np.log(temp1)
        error = error+temp2
    error_final = error/n
    
    return error_final

def logistic_gradient(w, X, y):

    # compute the gradient of the log-loss error (vector) with respect
    # to w (vector) for the given data X and y  
    #
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = d length gradient vector (not a d x 1 matrix)

    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    gradient = 0
    n= len(X)
    
    for i in range(n):
        under_exp = math.exp(y[i]*(np.dot(X[i,:],w)))
        temp1 = y[i]/(1+ np.exp(under_exp))
        temp2 = X[i]*temp1
        gradient = gradient+temp2
    gradient = -(gradient/n)
                  
    return gradient

def run_gradient_descent(X,y):
    old_w = np.array([-1]*X.shape[1])
    # change this value! This is an unreasonable step size
    step_size = 0.03
    new_w = old_w - 1
    print(new_w)
    while ((new_w-old_w)**2).sum() > .0000000001:
        temp = new_w
        s_k = logistic_gradient(old_w,X,y)
        new_w = old_w - (step_size*s_k)
        print(((new_w-old_w)**2).sum())
        old_w = temp
    return new_w


In [ ]:
def logistic_objective(w, X, y):

    # compute log-loss error (scalar) with respect
    # to w (vector) for the given data X and y                               
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = scalar
    
    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    error = 0
    loss = 0
    import math as m
    for i in range(len(X)):
        loss+= m.log(1+ m.exp(-y[i]*np.dot(w,X[i])))
        
    error = loss/len(X)
    return error

def logistic_gradient(w, X, y):

    # compute the gradient of the log-loss error (vector) with respect
    # to w (vector) for the given data X and y  
    #
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = d length gradient vector (not a d x 1 matrix)
    import math as m
    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    grad = 0
    for i in range(len(X)):
        grad+= y[i]/(1+ m.exp(y[i]*np.dot(w,X[i])))*X[i]
             
    gradient = -1*(grad/len(X))
    
    return gradient

def run_gradient_descent(X,y):
    old_w = np.array([-1]*X.shape[1])
    # change this value! This is an unreasonable step size
    step_size = 0.03
    new_w =old_w - 1
    print(new_w)
    while ((new_w-old_w)**2).sum() > .0000000001:
        temp = new_w
        new_w = old_w - step_size*logistic_gradient(old_w,X,y)
        old_w = temp
    return new_w

In [ ]:
from scipy.stats import uniform, bernoulli
import functools
draw_binary = functools.partial(np.random.binomial,n=1)

## Simulated data to test your method
DATA_SIZE = 10000
x1 = bernoulli(.5).rvs(DATA_SIZE)
x2 = np.floor(uniform(18,60).rvs(DATA_SIZE))
true_w = [-9, 3.5, 0.2]
xb = true_w[0] + true_w[1]*x1 + true_w[2]*x2
p = 1/(1 + np.exp(-xb))
y = np.array([1 if draw_binary(p=v) else -1 for v in p])

In [ ]:
from sklearn.linear_model import LogisticRegression

# notice that logistic regression as implemented in sklearn does not get the exact results either!
# so you shouldn't worry if you're a bit off
X = np.hstack([np.ones((len(xb),1)), x1[:,np.newaxis], x2[:,np.newaxis]])
model = LogisticRegression(solver='liblinear', random_state=0,fit_intercept=False)
model.fit(X,y).coef_

array([[-8.45776638,  3.26111601,  0.18966203]])

In [ ]:
# this is how we will test your results
gd_result = run_gradient_descent(X,y)
# is your result relatively close to the truth?
np.abs(true_w-gd_result).sum()

Streaming output truncated to the last 5000 lines.
[-10.3845856    2.15681696   0.23531256]
[-10.375213     2.20981687   0.16418421]
[-10.3832269    2.15849609   0.27178633]
[-10.36553435   2.21513617   0.61734064]
[-10.38449429   2.15901213   0.20596704]
[-10.37501764   2.21241384   0.32449007]
[-10.3803259    2.1619232    0.36782351]
[-10.37902544   2.21168924   0.1678613 ]
[-10.38590294   2.16053303   0.16621278]
[-10.36991953   2.2167466    0.58827774]
[-10.37644363   2.16579302   0.60611374]
[-10.37926822   2.21404477   0.29810168]
[-10.38587704   2.16307944   0.31424791]
[-10.38205551   2.21385993   0.18001744]
[-10.38938537   2.16259702   0.17281334]
[-10.3747426    2.21811383   0.50150254]
[-10.38093659   2.16739819   0.55569035]
[-10.38330636   2.21558718   0.22767077]
[-10.39005947   2.16474081   0.2700862 ]
[-10.38135258   2.21749251   0.28977152]
[-10.39121467   2.16530423   0.20834991]
[-10.38370684   2.21749814   0.18615674]
[-10.38729569   2.16810275   0.35844666]
[-10.3

##  Part 2.2 - Optimization with Newton-Raphson

In Part 2.2, you are going to, instead of using gradient descent, use the Newton-Raphson method to optimize the same logistic regression model. To do so, you will need to 1) implement the `logistic_hessian` function to compute the Hessian matrix of logistic loss for the given data set, and 2) use `scipy`'s `optimize` function to perform the optimization, rather than writing a function by hand to do so.  

For Part 2.2, you will only need to implement these functions, we will test them using our own code. You can, however, perform the same kinds of tests that we proposed above to check your work! 

In [ ]:
def logistic_hessian(w, X, y):

    # compute the Hessian of the log-loss error (matrix) with respect
    # to w (vector) for the given data X and y                               
    #
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # Hessian = d x d matrix
    
    
    hessian = 0
    if len(w.shape) == 1:
        w = w[:,np.newaxis]

    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    for i in range(len(X)):
        t = np.array(X[i,:])
        under_exp = math.exp(y[i]*(np.dot(np.transpose(w),X[i,:])))
        ps = under_exp**2
        t = t.reshape(3,1)
        hessian += (under_exp/ps)*t*(np.transpose(t))

    hessian = hessian/len(X)
    return hessian

In [ ]:
from scipy.optimize import minimize

def run_newton_raphson(X,y):
    args = (X,y[:,np.newaxis])
    opts = {'maxiter' : 50}    # Preferred value.    
    w_init = np.zeros(X.shape[1])
    
    # note: this is *almost* what you need, you just need to figure out what arguments are necessary here!
    soln = minimize(logistic_objective,
                    x0 = w_init,
                    args=args,
                    method='Newton-CG',
                    jac=logistic_gradient,
                    hess = logistic_hessian,
                    options=opts)

    w = np.transpose(np.array(soln.x))
    w = w[:,np.newaxis]
    return w


In [ ]:
run_newton_raphson(X,y)

array([[-5.62953335],
       [ 2.11240016],
       [ 0.1316571 ]])